## **Import libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.utils import get_file

import numpy as np
import os
import time

## **Download dataset**

In [2]:
# Download the Shakespeare dataset
path_to_file = get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

## **Read the data**

In [3]:
# Read the data and decode it
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
# To get the length of the dataset
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a peek at the dataset, to see how it looks and is structured
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# To get the unique characters in the data
vocab = sorted(set(text))
print(f'There are {len(vocab)} unique characters in the text')

There are 65 unique characters in the text


## **Process the text**

### Vectorise the text

In [7]:
# Convert each character into a numeric id
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
# Convert numeric id into characters
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
# Function to extract string from the characters converted from the numeric ids
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training data and targets

In [10]:
# Generate ids for all the tokens in the text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [11]:
# Convert the text vector into a stream of character indices
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [13]:
# To convert the individual characters to sequences of the desired size
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
# Function to take a sequence as input, duplicate and shift it to align the input and label for each timestep
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [15]:
dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

# Generate the training dataset
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

## **Build The Model**

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [19]:
# Define the model
class MyModel(tf.keras.Model):
  
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # The input layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # The RNN layer
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # The output layer
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [20]:
# Instantiate an RNN model object
model = MyModel(
    # vocabulary size must match the `StringLookup` layers
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## **Train the model**

### Attach an optimizer, and a loss function

In [21]:
# Apply sparse categorical crossentropy loss function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
# Compile with loss function and ADAM optimizer
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [23]:
# Directory to save the model checkpoints
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [24]:
# The number of epochs
EPOCHS = 20

In [25]:
# Train the model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 28s 135ms/step - loss: 2.6808
Epoch 2/20
172/172 [==============================] - 25s 133ms/step - loss: 1.9601
Epoch 3/20
172/172 [==============================] - 25s 134ms/step - loss: 1.6828
Epoch 4/20
172/172 [==============================] - 25s 134ms/step - loss: 1.5262
Epoch 5/20
172/172 [==============================] - 25s 135ms/step - loss: 1.4318
Epoch 6/20
172/172 [==============================] - 25s 134ms/step - loss: 1.3650
Epoch 7/20
172/172 [==============================] - 25s 134ms/step - loss: 1.3137
Epoch 8/20
172/172 [==============================] - 24s 132ms/step - loss: 1.2684
Epoch 9/20
172/172 [==============================] - 25s 134ms/step - loss: 1.2272
Epoch 10/20
172/172 [==============================] - 24s 133ms/step - loss: 1.1860
Epoch 11/20
172/172 [==============================] - 24s 133ms/step - loss: 1.1460
Epoch 12/20
172/172 [==============================] - 24s 133ms/step - lo

## **Generate text**

In [26]:
# Model to make a single step prediction
class OneStep(tf.keras.Model):
  
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [27]:
# Instantiate model object
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [28]:
# Run the model in a loop to generate text
start = time.time()
states = None
# Initial character to start the model
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The doughbour of a fellow, she sons, of worship,
The ghost to clothe my foe, sir, and I
love, and a spirit, wine are glads:
Out, and all ready expiring Juliet,
She wilt say 'she's lawful, proud heart's less care!

BISHOP OF CARLISLE:
But then,' quoth he, a royal prince, a cover:
But as the same I none but seem to abbut;
But as in words are fortingling some coals
From who till she had made no kindred of the dispose:
But what makes BerNown shall you wounds;
For beauty makes thee here by life.
Ay, full as uncles no better; in which seem to vig
His goathy withing him for thee Duke of York.

GLOUCESTER:
On year'st or damned with a small-orver
as free my father's fault or little,
Oft child, 'tis gone: if heavens that thou reasons too much
The day were force even four a diftener me
To plain forth more advice: part of England,
Depair'd with slow of danger lies you.
For shame doth receive away yielder deliver!
O holy carefully hear me but shall swear I will,
For mother is but a statutes 

## **Export the Generator**

In [29]:
# Export the model
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets
